In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

In [2]:
songs = pd.read_csv(r"C:\Users\HP\Desktop\Internships\Letsgrow more\Tasks\Tasks1\Task1.3 datasets\songs.csv",nrows = 20000)
train = pd.read_csv(r"C:\Users\HP\Desktop\Internships\Letsgrow more\Tasks\Tasks1\Task1.3 datasets\train.csv", nrows = 20000)
members = pd.read_csv(r"C:\Users\HP\Desktop\Internships\Letsgrow more\Tasks\Tasks1\Task1.3 datasets\members.zip")
test = pd.read_csv(r"C:\Users\HP\Desktop\Internships\Letsgrow more\Tasks\Tasks1\Task1.3 datasets\test.csv")
submission = pd.read_csv(r"C:\Users\HP\Desktop\Internships\Letsgrow more\Tasks\Tasks1\Task1.3 datasets\sample_submission.csv" , nrows = 20000)
song_info = pd.read_csv(r"C:\Users\HP\Desktop\Internships\Letsgrow more\Tasks\Tasks1\Task1.3 datasets\song_extra_info.csv" , nrows = 20000)

In [3]:
members.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613


In [4]:
submission.rename(columns = {'PassengerId':'id' } , inplace = True)
submission.rename(columns = {'Transported':'Target' } , inplace = True)
submission.head()

,id,Target
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


In [5]:
song_info.head()

,song_id,name,isrc
0,LP7pLJoJFBvyuUwvu+oLzjT+bI+UeBPURCecJsX1jjs=,我們,TWUM71200043
1,ClazTFnk6r0Bnuie44bocdNMM3rdlrq0bCGAsGUWcHE=,Let Me Love You,QMZSY1600015
2,u2ja/bZE3zhCGxvbbOB3zOoUjx27u40cf5g09UXMoKQ=,原諒我,TWA530887303
3,92Fqsy0+p6+RHe2EoLKjHahORHR1Kq1TBJoClW9v+Ts=,Classic,USSM11301446
4,0QFmz/+rJy1Q56C1DuYqT9hKKqi5TUqx0sN0IwvoHrw=,愛投羅網,TWA471306001


In [6]:
song_info.isnull().sum()

song_id       0
name          0
isrc       2702
dtype: int64

In [7]:
songs.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [8]:
songs.isnull().sum()

song_id            0
song_length        0
genre_ids        346
artist_name        0
composer        8382
lyricist       14332
language           0
dtype: int64

In [9]:
songs['genre_ids'].fillna(' ' , inplace = True)
songs['composer'].fillna(' ' , inplace = True)
songs['lyricist'].fillna(' ' , inplace = True)

In [10]:
songs.isnull().sum()

song_id        0
song_length    0
genre_ids      0
artist_name    0
composer       0
lyricist       0
language       0
dtype: int64

In [11]:
train.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [12]:
train.isnull().sum()

msno                    0
song_id                 0
source_system_tab      67
source_screen_name    576
source_type            50
target                  0
dtype: int64

In [13]:
# train = train.drop(["source_system_tab" , "source_screen_name" , "source_type"] , axis = 1)
train=train.drop(['source_system_tab','source_screen_name','source_type'],axis=1)
train.head()

,msno,song_id,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1


In [14]:
train.rename(columns={'msno':'user_id'},inplace=True)
train.head()

,user_id,song_id,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1


In [15]:
train.shape

(20000, 3)

#### Data preprocessing

In [16]:
df=train.merge(songs,on='song_id')

In [17]:
df=df.drop(['song_length','language'],axis=1)
df.head()

,user_id,song_id,target,genre_ids,artist_name,composer,lyricist
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=,1,1259,Desiigner,Sidney Selby| Adnan Khan,
1,hZyOA+0yqClPLt6uIEndf8fG8szH/95eKMbaxLE5z30=,3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=,1,1259,Desiigner,Sidney Selby| Adnan Khan,
2,0LhkakIQDn36HZXI6ClQSO7W7jkpZAy+9MvYgPOZGrA=,skehue/d/R59G71dXYpntDwdjRRPlweN3JE8g40TgZU=,1,458,莊心妍,鄭建浩,鄭建浩
3,MofmAMt7P8LIcF4+LLlcjyIhYUzmv13L/LRwYFxiGYE=,skehue/d/R59G71dXYpntDwdjRRPlweN3JE8g40TgZU=,1,458,莊心妍,鄭建浩,鄭建浩
4,U9Z+N+szYGJHTPMn/C0V7yIyIC24fDI0RDRWChXATkg=,skehue/d/R59G71dXYpntDwdjRRPlweN3JE8g40TgZU=,0,458,莊心妍,鄭建浩,鄭建浩


In [18]:
df=df.merge(song_info,on='song_id').drop('isrc',axis=1)
df.head()

,user_id,song_id,target,genre_ids,artist_name,composer,lyricist,name
0,Eaq21N7ijazDjnqU5v1jjWewJopGxgZxkKr4GIq7JTs=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (Naiwen Yang),黃建為,葛大為,離心力 (Centrifugal Force)
1,xOYfEnlbSceZPxSFui3yz91tAospqh+DrSS18Ix4GPE=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,0,458,楊乃文 (Naiwen Yang),黃建為,葛大為,離心力 (Centrifugal Force)
2,p/hZuEXU9ExH9LKkBfIueZUaNVX9SXQC3JXA8MiNWi4=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (Naiwen Yang),黃建為,葛大為,離心力 (Centrifugal Force)
3,0m6KFYHZ0sEqA+UJ165welDpnMHu92JJYQ8sz1nChPU=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (Naiwen Yang),黃建為,葛大為,離心力 (Centrifugal Force)
4,f8TJLlsG/vc/A+mdobHDauYO3QFAZCiY1fnmYooXsOk=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (Naiwen Yang),黃建為,葛大為,離心力 (Centrifugal Force)


In [19]:
df.rename(columns={'name':'song_name'},inplace=True)
df.head()

,user_id,song_id,target,genre_ids,artist_name,composer,lyricist,song_name
0,Eaq21N7ijazDjnqU5v1jjWewJopGxgZxkKr4GIq7JTs=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (Naiwen Yang),黃建為,葛大為,離心力 (Centrifugal Force)
1,xOYfEnlbSceZPxSFui3yz91tAospqh+DrSS18Ix4GPE=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,0,458,楊乃文 (Naiwen Yang),黃建為,葛大為,離心力 (Centrifugal Force)
2,p/hZuEXU9ExH9LKkBfIueZUaNVX9SXQC3JXA8MiNWi4=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (Naiwen Yang),黃建為,葛大為,離心力 (Centrifugal Force)
3,0m6KFYHZ0sEqA+UJ165welDpnMHu92JJYQ8sz1nChPU=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (Naiwen Yang),黃建為,葛大為,離心力 (Centrifugal Force)
4,f8TJLlsG/vc/A+mdobHDauYO3QFAZCiY1fnmYooXsOk=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (Naiwen Yang),黃建為,葛大為,離心力 (Centrifugal Force)


#### Data Cleaning

In [20]:
df['genre_ids'].value_counts()

465            48
458            22
444            21
2022            2
921             2
2122            1
139|125|109     1
359             1
                1
1609            1
726             1
437             1
451             1
Name: genre_ids, dtype: int64

In [21]:
df['genre_ids']=df['genre_ids'].str.replace('|',' ',regex=True)
df['genre_ids'].value_counts()

465            48
458            22
444            21
2022            2
921             2
2122            1
139 125 109     1
359             1
                1
1609            1
726             1
437             1
451             1
Name: genre_ids, dtype: int64

In [22]:
df['artist_name']=df['artist_name'].str.replace('|',' ',regex=True)
df['artist_name']=df['artist_name'].str.replace('/',' ',regex=True)

df['composer']=df['composer'].str.replace('|',' ',regex=True)
df['composer']=df['composer'].str.replace('/',' ',regex=True)

df['lyricist']=df['lyricist'].str.replace('|',' ',regex=True)
df['lyricist']=df['lyricist'].str.replace('/',' ',regex=True)

In [23]:
df['artist_name']=df['artist_name'].str.lower()
df['composer']=df['composer'].str.lower()
df['lyricist']=df['lyricist'].str.lower()

In [24]:
df['songs_details']=df['artist_name']+''+df['composer']+df['lyricist']
df.head()

,user_id,song_id,target,genre_ids,artist_name,composer,lyricist,song_name,songs_details
0,Eaq21N7ijazDjnqU5v1jjWewJopGxgZxkKr4GIq7JTs=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (naiwen yang),黃建為,葛大為,離心力 (Centrifugal Force),楊乃文 (naiwen yang)黃建為葛大為
1,xOYfEnlbSceZPxSFui3yz91tAospqh+DrSS18Ix4GPE=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,0,458,楊乃文 (naiwen yang),黃建為,葛大為,離心力 (Centrifugal Force),楊乃文 (naiwen yang)黃建為葛大為
2,p/hZuEXU9ExH9LKkBfIueZUaNVX9SXQC3JXA8MiNWi4=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (naiwen yang),黃建為,葛大為,離心力 (Centrifugal Force),楊乃文 (naiwen yang)黃建為葛大為
3,0m6KFYHZ0sEqA+UJ165welDpnMHu92JJYQ8sz1nChPU=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (naiwen yang),黃建為,葛大為,離心力 (Centrifugal Force),楊乃文 (naiwen yang)黃建為葛大為
4,f8TJLlsG/vc/A+mdobHDauYO3QFAZCiY1fnmYooXsOk=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (naiwen yang),黃建為,葛大為,離心力 (Centrifugal Force),楊乃文 (naiwen yang)黃建為葛大為


In [25]:
df.user_id.value_counts()

gL8rPt5443Mn2vAVXbBuY7IB0haVmgkYC1VOK+p/PDM=    2
nqmpF78qWkpdYVDlHMdJxaIee39zLsesNuzmpPhdqiI=    2
Eaq21N7ijazDjnqU5v1jjWewJopGxgZxkKr4GIq7JTs=    1
+aVBg5Dxo76UZL++0OFJUVW4LFOkhccy/U0NeMisBLo=    1
ptrX1ALQ5+tv6u55q/XyaUZOwrQCj9U48QOhJIVJ2mg=    1
                                               ..
CE8JY6cIwQgGVJDFRIMmbnIwTtSX56iQ7d0i0phCK2Q=    1
FWaCKrzp+SStanbYKX79rjIBrwZvstpNLuFOlB4Uq+A=    1
ORkEvs/WO1JXVMmbfbUwxJX/gF7pP5LZf/flrUtCbm8=    1
em/KMt7bezbtHjWrq5e+fNysPp4sacXuB65dIIWSva8=    1
j2Sx5B7BrjqCiT3ZwWK4AvepwM14QEalhTPi2/sgdG4=    1
Name: user_id, Length: 101, dtype: int64

In [26]:
df.duplicated().sum()

0

In [27]:
#Creating a copy file before performing similarity
temp_df=df.copy()
temp_df.head()

,user_id,song_id,target,genre_ids,artist_name,composer,lyricist,song_name,songs_details
0,Eaq21N7ijazDjnqU5v1jjWewJopGxgZxkKr4GIq7JTs=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (naiwen yang),黃建為,葛大為,離心力 (Centrifugal Force),楊乃文 (naiwen yang)黃建為葛大為
1,xOYfEnlbSceZPxSFui3yz91tAospqh+DrSS18Ix4GPE=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,0,458,楊乃文 (naiwen yang),黃建為,葛大為,離心力 (Centrifugal Force),楊乃文 (naiwen yang)黃建為葛大為
2,p/hZuEXU9ExH9LKkBfIueZUaNVX9SXQC3JXA8MiNWi4=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (naiwen yang),黃建為,葛大為,離心力 (Centrifugal Force),楊乃文 (naiwen yang)黃建為葛大為
3,0m6KFYHZ0sEqA+UJ165welDpnMHu92JJYQ8sz1nChPU=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (naiwen yang),黃建為,葛大為,離心力 (Centrifugal Force),楊乃文 (naiwen yang)黃建為葛大為
4,f8TJLlsG/vc/A+mdobHDauYO3QFAZCiY1fnmYooXsOk=,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (naiwen yang),黃建為,葛大為,離心力 (Centrifugal Force),楊乃文 (naiwen yang)黃建為葛大為


In [28]:
temp_df.songs_details.duplicated().sum()

54

In [29]:
temp_df.shape

(103, 9)

In [30]:
temp_df.duplicated().sum()

0

In [31]:
temp_df=temp_df.drop('user_id',axis=1)

In [32]:
temp_df.head()

,song_id,target,genre_ids,artist_name,composer,lyricist,song_name,songs_details
0,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (naiwen yang),黃建為,葛大為,離心力 (Centrifugal Force),楊乃文 (naiwen yang)黃建為葛大為
1,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,0,458,楊乃文 (naiwen yang),黃建為,葛大為,離心力 (Centrifugal Force),楊乃文 (naiwen yang)黃建為葛大為
2,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (naiwen yang),黃建為,葛大為,離心力 (Centrifugal Force),楊乃文 (naiwen yang)黃建為葛大為
3,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (naiwen yang),黃建為,葛大為,離心力 (Centrifugal Force),楊乃文 (naiwen yang)黃建為葛大為
4,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (naiwen yang),黃建為,葛大為,離心力 (Centrifugal Force),楊乃文 (naiwen yang)黃建為葛大為


In [33]:
temp_df.duplicated().sum()

47

In [57]:
temp_df=temp_df.drop_duplicates()
temp_df

,index,song_id,target,genre_ids,artist_name,composer,lyricist,song_name,songs_details
0,0,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,1,458,楊乃文 (naiwen yang),黃建為,葛大為,離心力 (Centrifugal Force),楊乃文 (naiwen yang)黃建為葛大為
1,1,BwDDym6Tp7EnGudWBGJyjONZeX0/4ndu5PETiPWzav8=,0,458,楊乃文 (naiwen yang),黃建為,葛大為,離心力 (Centrifugal Force),楊乃文 (naiwen yang)黃建為葛大為
2,16,x/sJa7GF5nZ36aAl5lDSjSJPVlobOx6ROZ3qB33RhMY=,0,465,蔡健雅 (tanya chua),蔡健雅,蔡健雅,Letting Go,蔡健雅 (tanya chua)蔡健雅蔡健雅
3,17,x/sJa7GF5nZ36aAl5lDSjSJPVlobOx6ROZ3qB33RhMY=,1,465,蔡健雅 (tanya chua),蔡健雅,蔡健雅,Letting Go,蔡健雅 (tanya chua)蔡健雅蔡健雅
4,23,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,1,444,blackpink,teddy future bounce bekuh boom,teddy,PLAYING WITH FIRE,blackpinkteddy future bounce bekuh boomteddy
5,26,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,0,444,blackpink,teddy future bounce bekuh boom,teddy,PLAYING WITH FIRE,blackpinkteddy future bounce bekuh boomteddy
6,44,v27+O2w69lwrw/luclRexIr/mdGNCJJp/EBhmsLje40=,1,465,吳克群 (kenji wu),吳克羣,吳克羣,全部都給你,吳克群 (kenji wu)吳克羣吳克羣
7,46,v91olyybN9ksMslVNo8ZdbOwj/gtBv4VzK35LEk0B/M=,1,465,周杰倫 (jay chou),周杰倫,方文山,園遊會,周杰倫 (jay chou)周杰倫方文山
8,49,9TxeC10erd6a5ATnCa57MLZrTkgNgG7zbJaHJryURFg=,1,2122,diana krall,,,'S Wonderful,diana krall
9,50,R8o3H0gQix1FHU1wk8n8QQOXDMYp/Hj5Hshmcm9RA6s=,1,2022,bon jovi,jon bon jovi richie sambora brett james,,What Do You Got?,bon jovijon bon jovi richie sambora brett ja...


In [58]:
temp_df.reset_index(inplace=True)

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(analyzer='word',stop_words='english')
tfidf_matrix=tfidf.fit_transform(temp_df['songs_details'])

In [60]:
tfidf_matrix


<56x169 sparse matrix of type '<class 'numpy.float64'>'
	with 209 stored elements in Compressed Sparse Row format>

In [61]:
temp_df.shape

(56, 10)

In [62]:
temp_df.reset_index(drop=True, inplace=True)

#### Building Similarity

In [63]:
from sklearn.metrics.pairwise import cosine_similarity

In [64]:
cosine_similarities=cosine_similarity(tfidf_matrix)

In [65]:
cosine_similarities

array([[1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [66]:
sorted(list(enumerate(cosine_similarities[0])),reverse=True,key=lambda x:x[1])[1:6]

[(1, 1.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0)]

In [67]:
#In which you can recommend only index
def recommend(song):
    song_index=temp_df[temp_df['song_name']==song].index[0]
    distances=cosine_similarities[song_index]
    song_list=sorted(list(enumerate(cosine_similarities[0])),reverse=True,key=lambda x:x[1])[1:6]
    for i in song_list:
        print(i[0])

#### User based Recommender-Content

In [82]:
def recommend(song):
    song_index=temp_df[temp_df['song_name']==song].index[0]
    distances=cosine_similarities[song_index]
    song_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:4]
                                                                               
    for i in song_list:
        print(temp_df.iloc[i[0]].song_name)
        